In [146]:
import pandas as pd
import numpy as np
import re #  Python의 정규 표현식(regular expression) 모듈 -> re.sub() 함수를 사용하여 문자열에서 정규 표현식 패턴에 일치하는 부분을 다른 문자열로 대체
import html # html 엔터티 코드 제거를 위해 사용
from sklearn.feature_extraction.text import TfidfVectorizer # sklearn 안에 있는 패키지
from sklearn.metrics.pairwise import cosine_similarity # TF-IDF 값을 기준으로 코사인 유사도를 통해 가장 유사한 제목을 찾아주기떄문에 cosine_similarity를 사용

#### 데이터 전처리 (특수문자 제거 및 null값 처리 후 csv파일로 저장)

In [145]:
# # 데이터를 데이터프레임으로 로드
# data = pd.read_csv("../library_book_data.csv", encoding='utf-8')# 데이터 파일 경로
# print(data)

       SEQ_NO  ISBN_THIRTEEN_NO VLM_NM  \
0     6352228     9791156759270    NaN   
1     6352229     9791168120877    NaN   
2     6352230     9791168120839    NaN   
3     6352231     9791168120846    NaN   
4     6352232     9791168120747    NaN   
...       ...               ...    ...   
1994  6354246     9791164670871    NaN   
1995  6354247     9788994229003    NaN   
1996  6354248     9788994027203    NaN   
1997  6354249     9791156759126    NaN   
1998  6354250     9788988963258    NaN   

                                          TITLE_NM                   AUTHR_NM  \
0     너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이                  이용신 (지은이)   
1            일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이                  차도하 (지은이)   
2                      본격 한중일 세계사 12 - 임오군란과 통킹 위기                굽시니스트 (지은이)   
3             즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전  올리비아 레메스 (지은이), 김잔디 (옮긴이)   
4                        오늘도 리추얼 : 음악, 나에게 선물하는 시간                  정혜윤 (지은이)   
...      

In [147]:
# # Null 값은 빈 문자열로 대체
# data.fillna("", inplace=True)
# print(data)

       SEQ_NO  ISBN_THIRTEEN_NO VLM_NM  \
0     6352228     9791156759270          
1     6352229     9791168120877          
2     6352230     9791168120839          
3     6352231     9791168120846          
4     6352232     9791168120747          
...       ...               ...    ...   
1994  6354246     9791164670871          
1995  6354247     9788994229003          
1996  6354248     9788994027203          
1997  6354249     9791156759126          
1998  6354250     9788988963258          

                                          TITLE_NM                   AUTHR_NM  \
0     너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이                  이용신 (지은이)   
1            일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이                  차도하 (지은이)   
2                      본격 한중일 세계사 12 - 임오군란과 통킹 위기                굽시니스트 (지은이)   
3             즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전  올리비아 레메스 (지은이), 김잔디 (옮긴이)   
4                        오늘도 리추얼 : 음악, 나에게 선물하는 시간                  정혜윤 (지은이)   
...      

/tmp/ipykernel_67379/3805302243.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna("", inplace=True)


In [148]:
# # 모든 열에 대해 특수 문자와 HTML 엔터티 코드 모두 제거
# for col in data.columns:
#     data[col] = data[col].apply(lambda x: re.sub(r'[^\w\s]', '', html.unescape(str(x))))
# # 결과 출력
# print(data)

       SEQ_NO ISBN_THIRTEEN_NO VLM_NM                              TITLE_NM  \
0     6352228    9791156759270          너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이   
1     6352229    9791168120877         일기에도 거짓말을 쓰는 사람  99년생 시인의 자의식 과잉 에세이   
2     6352230    9791168120839                   본격 한중일 세계사 12  임오군란과 통킹 위기   
3     6352231    9791168120846          즉시 기분을 바꿔드립니다  신기하게 마음이 편해지는 응급 처방전   
4     6352232    9791168120747                      오늘도 리추얼  음악 나에게 선물하는 시간   
...       ...              ...    ...                                   ...   
1994  6354246    9791164670871                                 학교 서클대화가 필요해   
1995  6354247    9788994229003                             People Make City   
1996  6354248    9788994027203                                        동방해경표   
1997  6354249    9791156759126                             큰글자도서 그렇다면 정상입니다   
1998  6354250    9788988963258                                        그 사람들   

                  AUTHR_NM PUBLISHER_NM PBLICTE_DE 

In [149]:
# data.to_csv('../cleaned_data.csv', index=False, encoding='utf-8')

#### TF-IDF
사용해 텍스트 문서에서 각 단어의 중요성을 가중치로 나타내어 각 단어의 중요성을 나타내고, 주어진 문서 내에서 해당 단어가 얼마나 중요한지를 측정 (빈도수)

In [136]:
# 데이터를 데이터프레임으로 로드
data = pd.read_csv('../cleaned_data.csv', encoding='utf-8')# 데이터 파일 경로
print(data)

       SEQ_NO  ISBN_THIRTEEN_NO VLM_NM                              TITLE_NM  \
0     6352228     9791156759270    NaN   너에게 목소리를 보낼게  달빛천사 성우 이용신의 첫 번째 에세이   
1     6352229     9791168120877    NaN  일기에도 거짓말을 쓰는 사람  99년생 시인의 자의식 과잉 에세이   
2     6352230     9791168120839    NaN            본격 한중일 세계사 12  임오군란과 통킹 위기   
3     6352231     9791168120846    NaN   즉시 기분을 바꿔드립니다  신기하게 마음이 편해지는 응급 처방전   
4     6352232     9791168120747    NaN               오늘도 리추얼  음악 나에게 선물하는 시간   
...       ...               ...    ...                                   ...   
1997  6354246     9791164670871    NaN                          학교 서클대화가 필요해   
1998  6354247     9788994229003    NaN                      People Make City   
1999  6354248     9788994027203    NaN                                 동방해경표   
2000  6354249     9791156759126    NaN                      큰글자도서 그렇다면 정상입니다   
2001  6354250     9788988963258    NaN                                 그 사람들   

                  AUTHR_NM PUBLISHER_NM

In [137]:
# TF-IDF 벡터화기 생성
tfidf_vectorizer = TfidfVectorizer()

In [138]:
# TF-IDF 매트릭스 계산
tfidf_matrix = tfidf_vectorizer.fit_transform(data["TITLE_NM"])

In [139]:
print(tfidf_matrix) 

  (0, 4369)	0.33969744789722073
  (0, 3457)	0.33969744789722073
  (0, 4734)	0.3580529029310776
  (0, 3861)	0.3580529029310776
  (0, 2784)	0.3580529029310776
  (0, 3496)	0.3580529029310776
  (0, 3214)	0.3580529029310776
  (0, 2676)	0.3580529029310776
  (1, 2288)	0.3423392691675829
  (1, 4876)	0.3423392691675829
  (1, 4062)	0.32478936799351377
  (1, 330)	0.3423392691675829
  (1, 3652)	0.32478936799351377
  (1, 4175)	0.3123375149484095
  (1, 2122)	0.3423392691675829
  (1, 4800)	0.3423392691675829
  (1, 4369)	0.32478936799351377
  (2, 4631)	0.379849721093593
  (2, 5508)	0.4003747927958275
  (2, 4832)	0.4003747927958275
  (2, 39)	0.265006565852682
  (2, 3871)	0.379849721093593
  (2, 5720)	0.4003747927958275
  (2, 3524)	0.4003747927958275
  (3, 5300)	0.3557818093253376
  :	:
  (1993, 4022)	0.3853999783863399
  (1993, 1469)	0.3853999783863399
  (1993, 1714)	0.36564258523183835
  (1993, 1311)	0.3853999783863399
  (1993, 1418)	0.3516244917010496
  (1994, 1421)	0.7254573935342749
  (1994, 572)	0

In [140]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [141]:
print(cosine_sim)

[[1.         0.11033012 0.         ... 0.         0.         0.        ]
 [0.11033012 1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [142]:
# 추천할 도서 선택 (예시: 첫 번째 도서)
book_index = 550

In [143]:
# 해당 도서와 유사한 도서 추천 (상위 5개)
similar_books = list(enumerate(cosine_sim[book_index]))
similar_books.sort(key=lambda x: x[1], reverse=True)
recommended_books = similar_books[1:10]

In [144]:
# 추천된 도서 출력
for i, score in recommended_books:
    print(f"줄거리: {data.loc[i, 'TITLE_NM']}, 유사도: {score:.2f}")

줄거리: The first hundred years of painting in California 17751875     with biographical information and references relating to the artists, 유사도: 0.25
줄거리: Drawing in England from Hilliard to Hogarth Hardcover, 유사도: 0.17
줄거리: 호남의 현역작가들Established Honam Artists, 유사도: 0.16
줄거리: Alexander Calder 18981976 Hardcover F First Edition, 유사도: 0.16
줄거리: About place     recent art of the Americas, 유사도: 0.15
줄거리: Made in Korea Hardcover, 유사도: 0.15
줄거리: Woke Inc Inside Corporate Americas Social Justice Scam Hardcover, 유사도: 0.14
줄거리: The Western Photographs of John K Hillers Hardcover First Edition, 유사도: 0.14
줄거리: Master Drawings from the National Portrait Galleryfrom Elizabeth I to Elizabeth II Paperback, 유사도: 0.14
